In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy import signal
from scipy.signal import stft, cwt, morlet
import pywt

window_time = 30 # seconds
window_size = 30*window_time # sample rate of data is 30Hz
overlap = 0.8 # define a 80% overlap
step_size = int(window_size * (1 - overlap)) # calculate a step size of the moving window

print(f"The step size of each sample is {step_size}, this is determined via the overlap")

# directory = "data/DesktopActivity/ALL"
directory = "data/ReadingActivity"

The step size of each sample is 179, this is determined via the overlap


In [6]:
activity = "Magazine"
subject = "P01"

data = []
diff_data = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        if (subject in filename) and (activity in filename):
            df = pd.read_csv(os.path.join(directory, filename), header=None)
            
            # Calculate the differences
            df_diff = df.diff().fillna(0)
            
            for i in range(0, len(df) - window_size + 1, step_size):
                window = df.iloc[i : i + window_size].values
                data.append(window)
                
                diff_window = df_diff.iloc[i : i + window_size].values  # Get the corresponding window from df_diff
                diff_data.append(diff_window)
                
np_data = np.array(data)
np_diff_data = np.array(diff_data)  # Convert diff_data to a NumPy array
print(f"{np_data.shape}")
print(f"{np_diff_data.shape}")

x_cor = np_data[:, :, 0:1].squeeze()
y_cor = np_data[:, :, 1:2].squeeze()

print(f"{x_cor.shape}")
print(f"{y_cor.shape}")

x_diff = np_diff_data[:, :, 0:1].squeeze()  # x differences
y_diff = np_diff_data[:, :, 1:2].squeeze()  # y differences

# Perform Fourier Transform on each row
fft_x_cor = np.fft.fft(x_cor, axis=1)
fft_y_cor = np.fft.fft(y_cor, axis=1)
print(f"{fft_x_cor.shape}")
print(f"{fft_y_cor.shape}")


# Define the number of points per subplot and the number of subplots
points_per_subplot = window_size
num_subplots = len(np_data)

# Create a function to plot a specific subplot
def plot_subplot(i):
    start = i * points_per_subplot
    end = start + points_per_subplot

    # Create a figure with 2 subplots
    fig, axs = plt.subplots(6, figsize=(10, 10))

    # Plot x-coordinates
    axs[0].plot(x_cor[i, :], 'r-', label='True', alpha=0.5)
    axs[0].set_title('X-coordinates ({}-{})'.format(start, end))
    axs[0].legend()

    # Plot y-coordinates
    axs[1].plot(y_cor[i, :], 'r-', label='True', alpha=0.5)
    axs[1].set_title('Y-coordinates ({}-{})'.format(start, end))
    axs[1].legend()
    
    # Plot x-coordinate differences
    axs[2].plot(x_diff[i, :], 'b-', label='Difference', alpha=0.5)
    axs[2].set_title('X-coordinate Differences ({}-{})'.format(start, end))
    axs[2].legend()

    # Plot y-coordinate differences
    axs[3].plot(y_diff[i, :], 'b-', label='Difference', alpha=0.5)
    axs[3].set_title('Y-coordinate Differences ({}-{})'.format(start, end))
    axs[3].legend()
    
    # Plot Fourier Transform of x-coordinates
    axs[4].plot(np.abs(fft_x_cor[i, 1:]), 'g-', label='Fourier', alpha=0.5)
    # axs[4].plot(np.abs(fft_x_cor[i, 0:]), 'g-', label='Fourier', alpha=0.5)
    axs[4].set_title('Fourier Transform of X-coordinates')
    axs[4].legend()

    # Plot Fourier Transform of y-coordinates
    axs[5].plot(np.abs(fft_y_cor[i, 1:]), 'g-', label='Fourier', alpha=0.5)
    # axs[5].plot(np.abs(fft_y_cor[i, 0:]), 'g-', label='Fourier', alpha=0.5)
    axs[5].set_title('Fourier Transform of Y-coordinates')
    axs[5].legend()

    # Display the plots
    plt.tight_layout()
    plt.show()
    
    
# Create a slider to select the subplot
slider = widgets.IntSlider(min=0, max=num_subplots-1, step=1, description='Subplot:')

# Create an interactive plot viewer
widgets.interactive(plot_subplot, i=slider)


(146, 900, 2)
(146, 900, 2)
(146, 900)
(146, 900)
(146, 900)
(146, 900)


interactive(children=(IntSlider(value=0, description='Subplot:', max=145), Output()), _dom_classes=('widget-in…

In [7]:
activity = "Manga"
subject = "P01"

data = []
diff_data = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        if (subject in filename) and (activity in filename):
            df = pd.read_csv(os.path.join(directory, filename), header=None)
            
            # Calculate the differences
            df_diff = df.diff().fillna(0)
            
            for i in range(0, len(df) - window_size + 1, step_size):
                window = df.iloc[i : i + window_size].values
                data.append(window)
                
                diff_window = df_diff.iloc[i : i + window_size].values
                diff_data.append(diff_window)
                
np_data = np.array(data)
np_diff_data = np.array(diff_data)
print(f"{np_data.shape}")
print(f"{np_diff_data.shape}")

x_cor = np_data[:, :, 0:1].squeeze()
y_cor = np_data[:, :, 1:2].squeeze()

print(f"{x_cor.shape}")
print(f"{y_cor.shape}")

x_diff = np_diff_data[:, :, 0:1].squeeze()
y_diff = np_diff_data[:, :, 1:2].squeeze()

fft_x_cor = np.fft.fft(x_cor, axis=1)
fft_y_cor = np.fft.fft(y_cor, axis=1)
print(f"{fft_x_cor.shape}")
print(f"{fft_y_cor.shape}")

def plot_stft_for_sequence(i, data, fs=30, nperseg=None, noverlap=None, colormap='plasma'):
    sequence_length = data.shape[1]
    # Set nperseg to 1/8 of the sequence length if not provided
    if nperseg is None:
        nperseg = sequence_length // 64
    
    # Set noverlap to 75% of nperseg if not provided
    if noverlap is None:
        noverlap = int(0.75 * nperseg)
    
    f, t, Zxx = stft(data[i, :], fs=fs, nperseg=nperseg, noverlap=noverlap)
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud', cmap=colormap)
    plt.title(f'Spectrogram for Sequence {i+1}')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(label='Magnitude')
    plt.tight_layout()
    plt.show()
    
def plot_cwt_for_sequence(i, data, width_range=None, colormap='plasma'):
    if width_range is None:
        width_range = np.arange(1, 31)
    
    cwtmatr = cwt(data[i, :], morlet, width_range)
    plt.figure(figsize=(10, 4))
    plt.imshow(np.abs(cwtmatr), aspect='auto', extent=[0, len(data[i, :]), 1, 31], cmap=colormap, interpolation='bilinear')
    plt.colorbar(label='Magnitude')
    plt.ylabel('Scale (Width)')
    plt.xlabel('Time')
    plt.title(f'Wavelet Transform for Sequence {i+1}')
    plt.tight_layout()
    plt.show()
    


def plot_dwt_for_sequence(i, data, wavelet='db1', level=None, colormap='plasma'):
    coeffs = pywt.wavedec(data[i, :], wavelet, level=level)
    plt.figure(figsize=(10, 4))
    for j, coeff in enumerate(coeffs):
        plt.subplot(len(coeffs), 1, j+1)
        plt.plot(coeff)
        plt.ylabel(f'Level {j}')
    plt.xlabel('Time')
    plt.tight_layout()
    plt.show()


def interactive_plot(i):
    # plot_cwt_for_sequence(i, x_cor)
    # plot_cwt_for_sequence(i, y_cor)
    plot_dwt_for_sequence(i, x_cor)
    plot_dwt_for_sequence(i, y_cor)
    plot_stft_for_sequence(i, x_cor)
    plot_stft_for_sequence(i, y_cor)
    

# Create a slider to select the sequence
slider = widgets.IntSlider(min=0, max=len(np_data)-1, step=1, description='Sequence:')

# Create an interactive plot viewer
widgets.interactive(interactive_plot, i=slider)

# Create a slider to select the sequence
slider = widgets.IntSlider(min=0, max=len(np_data)-1, step=1, description='Sequence:')

# Create an interactive plot viewer
widgets.interactive(interactive_plot, i=slider)


(146, 900, 2)
(146, 900, 2)
(146, 900)
(146, 900)
(146, 900)
(146, 900)


interactive(children=(IntSlider(value=0, description='Sequence:', max=145), Output()), _dom_classes=('widget-i…